In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import multiprocessing
from torch import autograd
from fastai.conv_learner import *
from fastai.transforms import TfmType
from fasterai.transforms import *
from fasterai.images import *
from fasterai.dataset import *
from fasterai.visualize import *
from fasterai.callbacks import *
from fasterai.loss import *
from fasterai.modules import *
from fasterai.training import *
from fasterai.generators import *
from fastai.torch_imports import *
from pathlib import Path
from itertools import repeat
import tensorboardX
torch.cuda.set_device(3)
plt.style.use('dark_background')
torch.backends.cudnn.benchmark=True


In [3]:
IMAGENET = Path('data/imagenet/ILSVRC/Data/CLS-LOC/train')
OPENIMAGES = Path('data/openimages')
CIFAR10 = Path('data/cifar10/train')

proj_id = 'defade2'
TENSORBOARD_PATH = Path('data/tensorboard/' + proj_id)

#gpath = IMAGENET.parent/('bwc_withattn_sn_supertrain_retry_gen_96.h5')
#dpath = IMAGENET.parent/('bwc_withattn_sn_supertrain_retry_critic_96.h5')

c_lr=5e-4
c_lrs = np.array([c_lr,c_lr,c_lr])

g_lr=c_lr/5
g_lrs = np.array([g_lr/1000,g_lr/100,g_lr])

keep_pcts=[0.25,0.25]
gen_freeze_tos=[-1,0]

sn=True
self_attention=sn
lrs_unfreeze_factor=1.0

x_tfms = [RandomLighting(0.5, 0.5)]
extra_aug_tfms = []
torch.backends.cudnn.benchmark=True

## Training

In [4]:
netG = Unet34(nf_factor=2, self_attention=self_attention, sn=sn, leakyReLu=False).cuda()
#netGVis = ModelVisualizationHook(TENSORBOARD_PATH, netG, 'netG')
#load_model(netG, gpath)

netD = DCCritic(ni=3, nf=512, scale=16, self_attention=self_attention, sn=sn).cuda()
#netDVis = ModelVisualizationHook(TENSORBOARD_PATH, netD, 'netD')
#load_model(netD, dpath)

In [5]:
trainer = GANTrainer(netD=netD, netG=netG, genloss_fns=[FeatureLoss(multiplier=1e2)])
trainerVis = GANVisualizationHook(TENSORBOARD_PATH, trainer, 'trainer', jupyter=False, visual_iters=100)

In [6]:
scheds=[]

scheds.extend(GANTrainSchedule.generate_schedules(szs=[64,64], bss=[32,32], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms,keep_pcts=keep_pcts, 
    save_base_name=proj_id, c_lrs=c_lrs, g_lrs=g_lrs, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=gen_freeze_tos))

#unshock
scheds.extend(GANTrainSchedule.generate_schedules(szs=[96], bss=[16], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.1], 
    save_base_name=proj_id, c_lrs=c_lrs/20, g_lrs=g_lrs/20, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1]))

scheds.extend(GANTrainSchedule.generate_schedules(szs=[96,96], bss=[16,16], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=keep_pcts, 
    save_base_name=proj_id, c_lrs=c_lrs/2, g_lrs=g_lrs/2, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=gen_freeze_tos))



#unshock
scheds.extend(GANTrainSchedule.generate_schedules(szs=[128], bss=[12], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.1], 
    save_base_name=proj_id, c_lrs=c_lrs/30, g_lrs=g_lrs/30, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1]))

scheds.extend(GANTrainSchedule.generate_schedules(szs=[128,128], bss=[12,12], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=keep_pcts, 
    save_base_name=proj_id, c_lrs=c_lrs/3, g_lrs=g_lrs/3, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=gen_freeze_tos))


#unshock
scheds.extend(GANTrainSchedule.generate_schedules(szs=[192], bss=[4], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.1], 
    save_base_name=proj_id, c_lrs=c_lrs/80, g_lrs=g_lrs/80, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1]))

scheds.extend(GANTrainSchedule.generate_schedules(szs=[192], bss=[4], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.25], 
    save_base_name=proj_id, c_lrs=c_lrs/8, g_lrs=g_lrs/8, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1]))

scheds.extend(GANTrainSchedule.generate_schedules(szs=[192], bss=[4], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.25], 
    save_base_name=proj_id, c_lrs=c_lrs/16, g_lrs=g_lrs/16, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[0]))



#unshock
scheds.extend(GANTrainSchedule.generate_schedules(szs=[256], bss=[2], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.1], 
    save_base_name=proj_id, c_lrs=c_lrs/160, g_lrs=g_lrs/160, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1]))

scheds.extend(GANTrainSchedule.generate_schedules(szs=[256], bss=[2], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.25], 
    save_base_name=proj_id, c_lrs=c_lrs/16, g_lrs=g_lrs/16, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1]))

scheds.extend(GANTrainSchedule.generate_schedules(szs=[256], bss=[2], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.25], 
    save_base_name=proj_id, c_lrs=c_lrs/32, g_lrs=g_lrs/32, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[0]))

In [ ]:
trainer.train(scheds=scheds)

 37%|███▋      | 3544/9624 [56:39<1:00:41,  1.67it/s]
WDist 1.7886936664581299; RScore 0.9882144927978516; FScore 0.8004792332649231; GAddlLoss [2.59313]; Iters: 6580; GCost: -0.2394160032272339; GPenalty: [0]; ConPenalty: [0]
 37%|███▋      | 3564/9624 [56:55<48:50,  2.07it/s]
WDist 1.7609481811523438; RScore 0.9581296443939209; FScore 0.8028184771537781; GAddlLoss [2.60422]; Iters: 6590; GCost: 0.184540256857872; GPenalty: [0]; ConPenalty: [0]
 37%|███▋      | 3584/9624 [57:11<47:12,  2.13it/s]
WDist 1.7829782962799072; RScore 0.8625969886779785; FScore 0.9203812479972839; GAddlLoss [2.49485]; Iters: 6600; GCost: 0.1336802989244461; GPenalty: [0]; ConPenalty: [0]
 37%|███▋      | 3604/9624 [57:27<45:04,  2.23it/s]
WDist 2.044799327850342; RScore 0.7957980036735535; FScore 1.249001383781433; GAddlLoss [2.25803]; Iters: 6610; GCost: -0.058822985738515854; GPenalty: [0]; ConPenalty: [0]
 38%|███▊      | 3624/9624 [57:40<54:22,  1.84it/s]  
WDist 1.9641497135162354; RScore 1.080255746841